In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/multi-platform-online-courses-dataset/edx.csv
/kaggle/input/multi-platform-online-courses-dataset/skillshare.csv
/kaggle/input/multi-platform-online-courses-dataset/Udemy.csv
/kaggle/input/multi-platform-online-courses-dataset/Coursera.csv
/kaggle/input/edx-courses-dataset-2021/EdX.csv
/kaggle/input/dataset-of-1200-coursera-courses/edx.csv
/kaggle/input/dataset-of-1200-coursera-courses/Barkeley_extension.csv
/kaggle/input/dataset-of-1200-coursera-courses/Oxford.csv
/kaggle/input/dataset-of-1200-coursera-courses/Stanford.csv
/kaggle/input/dataset-of-1200-coursera-courses/udacity.csv
/kaggle/input/dataset-of-1200-coursera-courses/alison.csv
/kaggle/input/dataset-of-1200-coursera-courses/MIT ocw.csv
/kaggle/input/dataset-of-1200-coursera-courses/london school of economics.csv
/kaggle/input/dataset-of-1200-coursera-courses/coursera_update.csv
/kaggle/input/dataset-of-1200-coursera-courses/pluralsight.csv
/kaggle/input/dataset-of-1200-coursera-courses/futurelearn.csv
/kaggle/i

In [7]:
# Install NLTK
!pip list | grep nltk
! pip install -U kaleido
import nltk

nltk.download('punkt')  
nltk.download('wordnet')  

# Unzip per this stackoverflow: https://stackoverflow.com/questions/73849624/getting-error-while-submitting-notebook-on-kaggle-even-after-importing-nltk-libr
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

nltk                               3.2.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 21.6 MB/s eta 0:00:00:00:0100:01
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/co

In [51]:
# Setup

import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re

# Clean text 
def clean_text(text):
    lemma = WordNetLemmatizer() # lemmatizer
    text = re.sub("[^A-Za-z0-9 ]", "", text)
    text = text.lower()
    tokens = word_tokenize(text) # look into this tokenization
    tokens = [lemma.lemmatize(word) for word in tokens # lemmatize words and remove stopwords 
                if word not in stopwords.words("english")]
    return tokens


# Get the sentence embeddings for each course and user input with this function
# First get the word embeddings and average them out for the sentence (aka course/input)
# overall embedding

def get_document_embedding(doc, model):
    embeddings = [model.wv[word] for word in doc if word in model.wv] # Get individual embeddings into a list
    # Consider implementing exception handling 
    if len(embeddings) > 0:
        return np.mean(embeddings, axis=0) 
    else:
        return np.zeros(model.vector_size)

# Use previous functions to process user input into vector and use cosine 
# Similarity to find the most related courses
def recommend_courses(user_input, document_embeddings, data, top_n=5):
    cleaned_input = clean_text(user_input)
    input_embedding = get_document_embedding(cleaned_input, model)
    similarities = cosine_similarity([input_embedding], document_embeddings)[0]
    top_indices = np.argsort(similarities)[-top_n:][::-1]
    recommendations = data.iloc[top_indices][['name', 'topic', 'link', 'provider']]
    return recommendations

        

In [38]:
# Experiment with other data
# edx, coursera, harvard, mit ocw
# https://sparkbyexamples.com/pandas/pandas-read-multiple-csv-files/#:~:text=Load%20each%20file%20into%20individual,each%20file%20individually%20if%20needed.

# Normalize/clean course data to the name, topic, link, text format for now

dataMit = pd.read_csv("/kaggle/input/dataset-of-1200-coursera-courses/MIT ocw.csv")
# print(dataMit.head())
dataMit.columns = map(str.lower, dataMit.columns)
dataMit.rename(columns={'name ': 'name'}, inplace=True)
dataMit.rename(columns={'course link': 'link'}, inplace=True)
dataMit = dataMit[['name', 'topic', 'link']]
dataMit['text'] = dataMit['name'] + " " + dataMit['topic'] 
dataMit['provider'] = 'Massachussets Institute of Technology'

dataHarvard = pd.read_csv("/kaggle/input/dataset-of-1200-coursera-courses/Harvard_university.csv")
# print(dataHarvard.head())
dataHarvard.columns = map(str.lower, dataHarvard.columns)
dataHarvard.rename(columns={'link to course': 'link', 'about': 'topic'}, inplace=True)
dataHarvard = dataHarvard[dataHarvard['price'] == 'Free']
dataHarvard = dataHarvard[['name', 'topic', 'link']]
dataHarvard['text'] = dataHarvard['name'] + " " + dataHarvard['topic'] 
dataHarvard['provider'] = 'Harvard University'

dataEdx = pd.read_csv("/kaggle/input/edx-courses-dataset-2021/EdX.csv")
# print(dataEdx.head())
dataEdx.columns = map(str.lower, dataEdx.columns)
dataEdx["topic"] = dataEdx['about'] + '. ' + dataEdx['course description']
dataEdx["provider"] = 'edX - ' + dataEdx['university']
dataEdx = dataEdx[['name', 'topic', 'link', 'provider']]
dataEdx['text'] = dataEdx['name'] + " " + dataEdx["topic"]

# Coursera
dataCoursera = pd.read_csv("/kaggle/input/coursera-free-courses-dataset/coursera.csv")
dataCoursera.rename(columns={
    'title': 'name',
    'skills': 'topic',
    'url': 'link',
}, inplace=True)
dataCoursera = dataCoursera[dataCoursera['price'] == 'Free']

dataCoursera['text'] = dataCoursera['name'] + " " + np.where(pd.notna(dataCoursera['topic']), dataCoursera['topic'], "")

dataCoursera['provider'] = 'Coursera - ' + dataCoursera['course_by']
dataCoursera = dataCoursera[['name', 'topic', 'link', 'text', 'provider']]
dataCoursera = dataCoursera.fillna("") # Fill null values

print(dataCoursera.head())

                                  name  \
0            The Science of Well-Being   
1                    Indigenous Canada   
2                    Financial Markets   
3  The Science of Well-Being for Teens   
4           Introduction to Psychology   

                                               topic  \
0  Personal Development, Business Psychology, Ent...   
1  Leadership and Management, Business Psychology...   
2  Finance, Banking, Investment Management, Risk ...   
3                                                      
4  Business Psychology, Emotional Intelligence, W...   

                                                link  \
0  https://www.coursera.org/learn/the-science-of-...   
1   https://www.coursera.org/learn/indigenous-canada   
2  https://www.coursera.org/learn/financial-marke...   
3  https://www.coursera.org/learn/the-science-of-...   
4  https://www.coursera.org/learn/introduction-ps...   

                                                text  \
0  The Science of

In [60]:
# Create a Combined Dataframe
data = pd.concat([dataMit, dataHarvard, dataEdx, dataCoursera])
data.head()

,name,topic,link,text,provider
0,Energy Economics,"Science, Economics, Social Science",https://ocw.mit.edu/courses/14-44-energy-econo...,"Energy Economics Science, Economics, Social Sc...",Massachussets Institute of Technology
1,Identity and Difference,"Social Science, Society, Anthropology",https://ocw.mit.edu/courses/21a-218j-identity-...,"Identity and Difference Social Science, Societ...",Massachussets Institute of Technology
2,Single Variable Calculus,"Mathematics, Differential Equations, Calculus",https://ocw.mit.edu/courses/18-01-single-varia...,"Single Variable Calculus Mathematics, Differen...",Massachussets Institute of Technology
3,Libertarianism in History,"Humanities, History, Social Science",https://ocw.mit.edu/courses/21h-181-libertaria...,"Libertarianism in History Humanities, History,...",Massachussets Institute of Technology
4,Introductory Biology,"Science, Health and Medicine, Biology",https://ocw.mit.edu/courses/7-016-introductory...,"Introductory Biology Science, Health and Medic...",Massachussets Institute of Technology


In [61]:
# The list of tokenized sentences, ie our Corpus 
data['cleaned_text'] = data['text'].apply(clean_text) # Add clean text column to dataframe
data['cleaned_text'].head()

0    [energy, economics, science, economics, social...
1    [identity, difference, social, science, societ...
2    [single, variable, calculus, mathematics, diff...
3    [libertarianism, history, humanity, history, s...
4    [introductory, biology, science, health, medic...
Name: cleaned_text, dtype: object

In [62]:
# Corpus = list of tokenized sentences (already cleaned)
corpus = data['cleaned_text'].tolist()
print(corpus[0])

# Train Word2Vec
model = Word2Vec(
    sentences=corpus,
    vector_size=40,
    window=5,       # Larger window for broader context
    min_count=5,    # Ignore very rare words
    workers=4,
    epochs=10
)

['energy', 'economics', 'science', 'economics', 'social', 'science']


In [63]:
# Get a list of the document embedding vector for each sentence in the cleaned text data. The indices will be aligned with the original course rows in dataframe
document_embeddings = [get_document_embedding(doc, model)
                      for doc in data['cleaned_text']]
print(f'list of sentence vectors/sentences: {len(document_embeddings)}')
print(f'each sentence has {document_embeddings[0].shape} dimensions')

list of sentence vectors/sentences: 4338
each sentence has (40,) dimensions


In [65]:
# User interface (abstracted away)
user_input = "data and python"
recommendations = recommend_courses(user_input, document_embeddings, data)
recommendations.head()

,name,topic,link,provider
590,用Python玩转数据 Data Processing Using Python,"Computer Programming, Python Programming, Stat...",https://www.coursera.org/learn/hipython,Coursera - Nanjing University
113,Data Science: R Basics,Build a foundation in R and learn how to wrang...,https://pll.harvard.edu/course/data-science-r-...,Harvard University
71,Statistics and R,An introduction to basic statistical concepts ...,https://pll.harvard.edu/course/statistics-and-r,Harvard University
596,Базы данных и SQL в обработке и анализе данных,"Data Management, Databases, SQL, Statistical P...",https://www.coursera.org/learn/sql-data-scienc...,Coursera - IBM Skills Network
174,Python Data Structures,The second course in Python for Everybody expl...,https://www.edx.org/course/python-data-structures,edX - The University of Michigan


# Important Notes

* Original Source: https://www.kaggle.com/code/shtrausslearning/nlp-edx-course-recommendations
* DeepSeek AI for original base which also calls the document embeddings for the input
* I have added comments to better study and understand the code as a base to build off of
* Instead of directly matching to a course index in the dataset which limits the use of the model

# Todo (to better understand and be able to present on this topic)

* Study cosine similarity\
* Word2Vec and stopwords/lemmatization
* Coursera, futurelearn, udemy
* Build a frontend for the app

# Improvements

reshaped corpus as list instead of series, increased context window, 